In [9]:
import os
import json
import pandas as pd

# Initialize the master DataFrame to store all results
master_df = pd.DataFrame()

# Define the directory containing the JSON files
directory_path = "C:\\data for lol\\games"

# Loop over each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        with open(os.path.join(directory_path, filename), 'r') as file:
            loaded_data = json.load(file)

        # Extract the platformGameId
        platformGameId = None
        for entry in loaded_data:
            if entry.get("eventType") == "game_info":
                platformGameId = entry.get("platformGameId")
                break

        # Process the loaded data
        results = {
            'platformGameId': platformGameId,
            'building_destroyed': {},
            'champion_kill': {},
            'baron_kill': {},
            'riftHerald_kill': {},
            'dragon_kill': {},
            'game_end': {},
            'first_baron_team': None,
            'first_riftHerald_team': None
        }

        first_baron_time = float('inf')
        first_riftHerald_time = float('inf')

        for event in loaded_data:
            event_type = event.get("eventType")
            if event_type == "building_destroyed":
                team_id = event["teamID"]
                results['building_destroyed'][team_id] = results['building_destroyed'].get(team_id, 0) + 1
            elif event_type == "champion_kill":
                killer_team_id = event["killerTeamID"]
                results['champion_kill'][killer_team_id] = results['champion_kill'].get(killer_team_id, 0) + 1
            elif event_type == "epic_monster_kill":
                monster_type = event.get("monsterType")
                killer_team_id = event["killerTeamID"]
                if monster_type == "baron":
                    results['baron_kill'][killer_team_id] = results['baron_kill'].get(killer_team_id, 0) + 1
                    if event["gameTime"] < first_baron_time:
                        first_baron_time = event["gameTime"]
                        results['first_baron_team'] = killer_team_id
                elif monster_type == "riftHerald":
                    results['riftHerald_kill'][killer_team_id] = results['riftHerald_kill'].get(killer_team_id, 0) + 1
                    if event["gameTime"] < first_riftHerald_time:
                        first_riftHerald_time = event["gameTime"]
                        results['first_riftHerald_team'] = killer_team_id
                elif monster_type == "dragon":
                    results['dragon_kill'][killer_team_id] = results['dragon_kill'].get(killer_team_id, 0) + 1
            elif event_type == "game_end":
                winning_team_id = event.get("winningTeam")
                results['game_end'][winning_team_id] = results['game_end'].get(winning_team_id, 0) + 1

        # Transform the results into a DataFrame
        data = {
            'platformGameId': [],
            'teamID': [],
            'building_destroyed': [],
            'destroyed_opponent_buildings': [],
            'champion_kill': [],
            'baron_kill': [],
            'riftHerald_kill': [],
            'dragon_kill': [],
            'first_baron_team': [],
            'first_riftHerald_team': [],
            'game_end': []
        }

        all_teams = set(list(results['building_destroyed'].keys()) + list(results['champion_kill'].keys()) +
                        list(results['baron_kill'].keys()) + list(results['riftHerald_kill'].keys()) +
                        list(results['game_end'].keys()) + list(results['dragon_kill'].keys()))

        for team_id in all_teams:
            data['platformGameId'].append(platformGameId)
            data['teamID'].append(team_id)
            data['building_destroyed'].append(results['building_destroyed'].get(team_id, 0))
            opponent_team_id = [tid for tid in all_teams if tid != team_id][0]  # Assuming only two teams
            data['destroyed_opponent_buildings'].append(results['building_destroyed'].get(opponent_team_id, 0))
            data['champion_kill'].append(results['champion_kill'].get(team_id, 0))
            data['baron_kill'].append(results['baron_kill'].get(team_id, 0))
            data['riftHerald_kill'].append(results['riftHerald_kill'].get(team_id, 0))
            data['dragon_kill'].append(results['dragon_kill'].get(team_id, 0))
            data['first_baron_team'].append(1 if team_id == results['first_baron_team'] else 0)
            data['first_riftHerald_team'].append(1 if team_id == results['first_riftHerald_team'] else 0)
            data['game_end'].append(results['game_end'].get(team_id, 0))

        df = pd.DataFrame(data)

        # Append the DataFrame to the master DataFrame using pd.concat
        master_df = pd.concat([master_df, df], ignore_index=True)

# Save the master DataFrame to a specific directory
output_path = "C:\\data for lol\\result chart\\consolidated_data.csv"
master_df.to_csv(output_path, index=False)


In [10]:


# Load the CSV
csv_data = pd.read_csv("C:\\data for lol\\result chart\\consolidated_data.csv")

# Load the JSON
with open("C:\data for lol\esports-data\mapping_data.json", 'r') as f:
    json_data = json.load(f)

# Create a mapping dictionary from the JSON data
mapping_dict = {entry['platformGameId']: entry['teamMapping'] for entry in json_data}

# Function to get the long official team ID using the platformGameId and teamID
def get_long_team_id(platformGameId, teamID):
    try:
        return mapping_dict[platformGameId][str(teamID)]
    except KeyError:
        return None

# Apply the function to get the long official team ID for each row
csv_data['long_official_team_id'] = csv_data.apply(lambda row: get_long_team_id(row['platformGameId'], row['teamID']), axis=1)


# Convert the 'long_official_team_id' column to string
csv_data['long_official_team_id'] = csv_data['long_official_team_id'].astype(str)

# Save the updated CSV data back to the original file
csv_data.to_csv("C:\\data for lol\\result chart\\consolidated_data.csv", index=False)




<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\15984\AppData\Local\Temp\ipykernel_8116\4086773657.py:5: SyntaxWarning: invalid escape sequence '\d'
  with open("C:\data for lol\esports-data\mapping_data.json", 'r') as f:


In [8]:


# Load the CSV data
csv_data = pd.read_csv("C:\\data for lol\\result chart\\consolidated_data.csv")

# Load the JSON data
with open("C:\\data for lol\\esports-data\\teams.json", 'r') as json_file:
    json_data = json.load(json_file)

# Create a dictionary with team_id as the key and name as the value
team_id_to_name = {team['team_id']: team['name'] for team in json_data}

# Map the long_official_team_id in the CSV data to the team names
csv_data['team_name'] = csv_data['long_official_team_id'].astype(str).map(team_id_to_name)

# Fill NaN values in the 'team_name' column with "Not Found"
csv_data['team_name'].fillna('Not Found', inplace=True)

# Save the updated CSV data (optional)
csv_data.to_csv("C:\\data for lol\\result chart\\consolidated_data.csv", index=False)
